In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import point
import folium as fol 
import geopy
import hvplot.pandas


In [49]:
dd_dat = pd.read_csv('doordash_data.csv')
dd_dat

,searched_zipcode,searched_lat,searched_lng,searched_address,searched_state,searched_city,searched_metro,city_slug,latitude,longitude,...,delivery_fee_raw,delivery_fee,delivery_time_raw,delivery_time,service_fee_raw,service_fee,phone,review_count,review_rating,RunDate
0,10003,40.732473,-73.987380,"41 Cooper Square, 7 E 7th St, New York, NY 100...",NY,New York,NY,NaN,40.675464,-73.980782,...,"$0.00 delivery fee, first order",0.0,24 - 34,29,NaN,NaN,NaN,1364,4.8,4/25/2022 7:01
1,11219,40.631075,-73.997695,"4701 15th Ave, Brooklyn, NY 11219, USA",NY,Brooklyn,NY,NaN,40.675464,-73.980782,...,"$0.00 delivery fee, first order",0.0,24 - 34,29,NaN,NaN,NaN,1364,4.8,4/25/2022 7:01
2,11216,40.678832,-73.950677,"300 Madison St, Brooklyn, NY 11216, USA",NY,Brooklyn,NY,NaN,40.675464,-73.980782,...,"$0.00 delivery fee, first order",0.0,24 - 34,29,NaN,NaN,NaN,1364,4.8,4/25/2022 7:01
3,11219,40.631075,-73.997695,"4701 15th Ave, Brooklyn, NY 11219, USA",NY,Brooklyn,NY,NaN,40.672978,-73.950462,...,"$0.00 delivery fee, first order",0.0,23 - 33,28,NaN,NaN,NaN,2000,4.6,4/25/2022 7:01
4,10003,40.732473,-73.987380,"41 Cooper Square, 7 E 7th St, New York, NY 100...",NY,New York,NY,NaN,40.672978,-73.950462,...,"$0.00 delivery fee, first order",0.0,23 - 33,28,NaN,NaN,NaN,2000,4.6,4/25/2022 7:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11214,2176,42.457335,-71.061662,"119 Myrtle St, Melrose, MA 02176, USA",MA,Melrose,BO,NaN,42.463513,-71.026401,...,"$0.00 delivery fee, first order",0.0,35 - 45,40,NaN,NaN,NaN,955,4.0,4/25/2022 7:01
11215,2176,42.457335,-71.061662,"119 Myrtle St, Melrose, MA 02176, USA",MA,Melrose,BO,NaN,42.478599,-71.063536,...,"$0.00 delivery fee, first order",0.0,46 - 56,51,NaN,NaN,NaN,82,4.1,4/25/2022 7:01
11216,2176,42.457335,-71.061662,"119 Myrtle St, Melrose, MA 02176, USA",MA,Melrose,BO,NaN,42.460613,-71.027119,...,"$0.00 delivery fee, first order",0.0,27 - 37,32,NaN,NaN,NaN,110,4.5,4/25/2022 7:01
11217,2176,42.457335,-71.061662,"119 Myrtle St, Melrose, MA 02176, USA",MA,Melrose,BO,NaN,42.463535,-71.026164,...,"$0.00 delivery fee, first order",0.0,29 - 39,34,NaN,NaN,NaN,23,3.7,4/25/2022 7:01


In [50]:
clean_dd = dd_dat.drop(['city_slug', 'delivery_fee_raw', 'delivery_fee', 'service_fee_raw', 'service_fee', 'phone', 'RunDate', 'delivery_time_raw'], axis=1)


In [51]:
clean_dd['cuisines split'] = clean_dd['cuisines'].str.split('|')
# clean_dd['cuisines'].loc[0].split("|")[0]
# t.split("|")
clean_dd.head(10)

,searched_zipcode,searched_lat,searched_lng,searched_address,searched_state,searched_city,searched_metro,latitude,longitude,distance,loc_name,loc_number,url,address,cuisines,delivery_time,review_count,review_rating,cuisines split
0,10003,40.732473,-73.987380,"41 Cooper Square, 7 E 7th St, New York, NY 100...",NY,New York,NY,40.675464,-73.980782,1.594482,Lotus Vietnamese Sandwiches,7552,https://www.doordash.com/store/7552/?pickup=false,"229 5th Avenue, Brooklyn, NY 11215, USA",Vietnamese|Bubble Tea|Smoothies|Sandwiches|Pho,29,1364,4.8,"[Vietnamese, Bubble Tea, Smoothies, Sandwiches..."
1,11219,40.631075,-73.997695,"4701 15th Ave, Brooklyn, NY 11219, USA",NY,Brooklyn,NY,40.675464,-73.980782,1.594482,Lotus Vietnamese Sandwiches,7552,https://www.doordash.com/store/7552/?pickup=false,"229 5th Avenue, Brooklyn, NY 11215, USA",Vietnamese|Bubble Tea|Smoothies|Sandwiches|Pho,29,1364,4.8,"[Vietnamese, Bubble Tea, Smoothies, Sandwiches..."
2,11216,40.678832,-73.950677,"300 Madison St, Brooklyn, NY 11216, USA",NY,Brooklyn,NY,40.675464,-73.980782,1.594482,Lotus Vietnamese Sandwiches,7552,https://www.doordash.com/store/7552/?pickup=false,"229 5th Avenue, Brooklyn, NY 11215, USA",Vietnamese|Bubble Tea|Smoothies|Sandwiches|Pho,29,1364,4.8,"[Vietnamese, Bubble Tea, Smoothies, Sandwiches..."
3,11219,40.631075,-73.997695,"4701 15th Ave, Brooklyn, NY 11219, USA",NY,Brooklyn,NY,40.672978,-73.950462,0.404577,Taqueria Milear,133345,https://www.doordash.com/store/133345/?pickup=...,"752 Nostrand Ave, Brooklyn, NY 11216, USA",Mexican|Tacos|Burritos|Dessert|Brunch,28,2000,4.6,"[Mexican, Tacos, Burritos, Dessert, Brunch]"
4,10003,40.732473,-73.987380,"41 Cooper Square, 7 E 7th St, New York, NY 100...",NY,New York,NY,40.672978,-73.950462,0.404577,Taqueria Milear,133345,https://www.doordash.com/store/133345/?pickup=...,"752 Nostrand Ave, Brooklyn, NY 11216, USA",Mexican|Tacos|Burritos|Dessert|Brunch,28,2000,4.6,"[Mexican, Tacos, Burritos, Dessert, Brunch]"
5,11216,40.678832,-73.950677,"300 Madison St, Brooklyn, NY 11216, USA",NY,Brooklyn,NY,40.672978,-73.950462,0.404577,Taqueria Milear,133345,https://www.doordash.com/store/133345/?pickup=...,"752 Nostrand Ave, Brooklyn, NY 11216, USA",Mexican|Tacos|Burritos|Dessert|Brunch,28,2000,4.6,"[Mexican, Tacos, Burritos, Dessert, Brunch]"
6,10003,40.732473,-73.987380,"41 Cooper Square, 7 E 7th St, New York, NY 100...",NY,New York,NY,40.674537,-73.950007,0.298770,Cielito Lindo,328460,https://www.doordash.com/store/328460/?pickup=...,"665 Nostrand Ave, Brooklyn, NY 11216, USA",Mexican|Breakfast|Burritos|Salads|Soup,25,163,4.7,"[Mexican, Breakfast, Burritos, Salads, Soup]"
7,11216,40.678832,-73.950677,"300 Madison St, Brooklyn, NY 11216, USA",NY,Brooklyn,NY,40.674537,-73.950007,0.298770,Cielito Lindo,328460,https://www.doordash.com/store/328460/?pickup=...,"665 Nostrand Ave, Brooklyn, NY 11216, USA",Mexican|Breakfast|Burritos|Salads|Soup,25,163,4.7,"[Mexican, Breakfast, Burritos, Salads, Soup]"
8,11219,40.631075,-73.997695,"4701 15th Ave, Brooklyn, NY 11219, USA",NY,Brooklyn,NY,40.674537,-73.950007,0.298770,Cielito Lindo,328460,https://www.doordash.com/store/328460/?pickup=...,"665 Nostrand Ave, Brooklyn, NY 11216, USA",Mexican|Breakfast|Burritos|Salads|Soup,25,163,4.7,"[Mexican, Breakfast, Burritos, Salads, Soup]"
9,10003,40.732473,-73.987380,"41 Cooper Square, 7 E 7th St, New York, NY 100...",NY,New York,NY,40.680930,-73.977140,1.394092,Artichoke Basille's Pizza,48755,https://www.doordash.com/store/48755/?pickup=f...,"59 5th Ave, Brooklyn, NY 11217, USA",Pizza|Pie,42,2000,4.5,"[Pizza, Pie]"


In [52]:
# f = fol.Figure(width=800, height=500)
# map=fol.Map(location=[33.977523191892004, -117.55304474566861], zoom_start=15).add_to(f)
# # map.add_child(fol.Marker(location=[33.977523191892004, -117.55304474566861],popup='BWW',icon=fol.Icon(color='red')))
# # map

# #
# #  mapx = pd.hvplot.points(33.977523191892004, -117.55304474566861, geo=True, tiles = True, alpha=0.70, frame_height=450, frame_width=800)
# # mapx

In [53]:
# Make an empty map
# Resize empty map
f = fol.Figure(width=800, height=500)

m = fol.Map(location=[38.319717, -99.548855], tiles="openstreetmap", zoom_start=4.4).add_to(f)

# create dataframe with lon, lat, and name of location
mdf = clean_dd[['loc_name', 'searched_lng', 'searched_lat']]
mdf = mdf.drop_duplicates()
mdf

# create markers with data from dataframe 
for i in range(0,len(mdf)):
   fol.Marker(
      location=[mdf.iloc[i]['searched_lat'], mdf.iloc[i]['searched_lng']],
      popup=mdf.iloc[i]['loc_name'],
   ).add_to(m)

# Show the map
# m

In [54]:
cleaner_dd = clean_dd.drop_duplicates(subset=['loc_name', 'latitude', 'longitude'], keep='last')

In [55]:
cleaner_dd.to_csv('cleaner_doordash_data.csv')

In [90]:
#map of customer locations
mapa = cleaner_dd.hvplot.points('searched_lng', 'searched_lat', geo=True, tiles = True, alpha=0.60, frame_height=450, frame_width=800, 
                            c='searched_state',  hover_cols=['loc_name'], s=100)
mapa

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [searched_lng,searched_lat]   (searched_state,loc_name)

In [80]:
#map of order location
mapb = cleaner_dd.hvplot.points('longitude', 'latitude', geo=True, tiles = True, frame_height=450, frame_width=800, 
                            c='searched_state',  hover_cols=['loc_name', 'delivery_time'], alpha=0.60, s=100)
mapb

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (searched_state,loc_name,delivery_time)

In [58]:
#combine all cuisine types into one list 
#going to try using a for loop.. there might be another way idk 

# all_cuisine_types = []
# for x in C: 
#     # print(x)
#     all_cuisine_types.extend(x)
#     # all_cuisine_types  = all_cuisine_types + x

# # all_cuisine_types = set(all_cuisine_types)

# all_cuisine_types
# C = cleaner_dd['cuisines split']
all_cuisine_types= cleaner_dd['cuisines split'].explode()

# set(all_cuisine_types)
all_cuisine_types.value_counts().head(25)


Sandwiches       1089
Salads           1069
Dessert           843
Chicken           658
Breakfast         584
Burgers           574
Seafood           455
Mexican           434
Pizza             432
Soup              428
Snacks            320
Tacos             313
American          312
Italian           299
Burritos          242
Fast Food         232
Fries             229
Noodles           227
Wraps             207
Chicken Wings     206
Smoothies         206
Coffee            203
Vegetarian        189
Ice Cream         176
Japanese          175
Name: cuisines split, dtype: int64

In [59]:
#refining definition for cuisine
int_cuisines = ['Indian', 
'Italian', 
'Japanese',
'Mexican',
'Thai', 
'Chinese', 
'French',
'Turkish',
'Greek',
'Arab',
'Spanish',
'Vietnamese',
'Korean',
'Malaysian',
'Russian',
'Ethiopian',
'Caribbean',
'African', 
'American']

all_cuisine_types = all_cuisine_types.reset_index()
all_cuisine_types = all_cuisine_types.drop(columns='index')
all_cuisine_types

# all_cuisine_types.values
dd_cuisines = {'cuisine': all_cuisine_types['cuisines split'].value_counts().index, 'count': all_cuisine_types['cuisines split'].value_counts().values}
dd_cuisines = pd.DataFrame(dd_cuisines)
dd_cuisines[dd_cuisines['cuisine'].isin(int_cuisines)]


,cuisine,count
7,Mexican,434
12,American,312
13,Italian,299
24,Japanese,175
27,Chinese,151
45,Thai,93
61,Indian,67
92,Caribbean,39
95,Korean,37
97,Vietnamese,37


In [60]:
NY = cleaner_dd['searched_metro'] == 'NY'

NY_temp = cleaner_dd['cuisines split'].where(NY)
NY_temp = NY_temp.explode().dropna()
NY_temp.value_counts()

NY_cuisine_types = NY_temp.reset_index()
NY_cuisine_types = NY_cuisine_types.drop(columns='index')
NY_cuisine_types

# all_cuisine_types.values
NY_cuisines = {'cuisine': NY_cuisine_types['cuisines split'].value_counts().index, 'count': NY_cuisine_types['cuisines split'].value_counts().values}
NY_cuisines = pd.DataFrame(NY_cuisines)
NY_cuisines[NY_cuisines['cuisine'].isin(int_cuisines)]

,cuisine,count
9,American,141
10,Mexican,119
14,Italian,106
29,Chinese,54
39,Japanese,45
45,Indian,37
46,Caribbean,36
50,Thai,30
94,Korean,14
147,Vietnamese,7


In [92]:
cleaner_dd['searched_metro'].value_counts()

NY    1620
LA     875
CH     848
BO     730
Name: searched_metro, dtype: int64

In [91]:
cleaner_dd['searched_state'].value_counts()

NY    1376
CA     875
IL     848
MA     730
NJ     244
Name: searched_state, dtype: int64

In [62]:
dd_dat['searched_city'].value_counts()

Brooklyn           2409
Chicago            1880
New York           1209
Boston              802
Los Angeles         501
Cambridge           384
Charlestown         383
Jersey City         372
Weehawken           371
Staten Island       366
Burbank             323
Glendale            321
Manhattan Beach     249
Torrance            246
Park Ridge          196
Glenview            191
Dorchester          185
Anaheim             175
Irvine              175
West Covina         172
La Grange            88
Astoria              84
Melrose              50
Revere               45
Belmont              42
Name: searched_city, dtype: int64

In [63]:
dd_dat['loc_name'].value_counts().head(10)

McDonald's            35
Walgreens             32
Burger King           32
CVS                   31
Subway                29
7-Eleven              26
The Ice Cream Shop    24
Baskin-Robbins        23
KFC                   23
Wendy's               22
Name: loc_name, dtype: int64

In [64]:
#get value counts of counts greater than a certain number 
dd_dat['loc_name'].value_counts().loc[lambda x : x >2]

McDonald's                   35
Walgreens                    32
Burger King                  32
CVS                          31
Subway                       29
                             ..
Curry Up Now                  3
Brooklyn Roasting Company     3
J’s Wong                      3
Wraps On Tap                  3
Gogi Grill                    3
Name: loc_name, Length: 3052, dtype: int64

In [65]:
#the #1 most ordered from places in each state and then do each city
#group by, then value counts
dd_dat.groupby('searched_state')['loc_name'].value_counts()

searched_state  loc_name                  
CA              McDonald's                    10
                Subway                         9
                7-Eleven                       8
                Baskin-Robbins                 8
                CVS                            8
                                              ..
NY              drippinsweet food truck        1
                family deli                    1
                honeygrow                      1
                nonna maria pizza              1
                pio pio cafe y restaurante     1
Name: loc_name, Length: 4434, dtype: int64

In [66]:
#figure out how to parse through cuisines and then group them getting a count of how many there are of each
#something like a new dataframe with each column as a cuisine type and they're grouped by store

In [67]:
#trying to figure out how to plot longitude and latitude 
dd_dat.longitude

0       -73.980782
1       -73.980782
2       -73.980782
3       -73.950462
4       -73.950462
           ...    
11214   -71.026401
11215   -71.063536
11216   -71.027119
11217   -71.026164
11218   -71.064515
Name: longitude, Length: 11219, dtype: float64

In [68]:
dd_dat['url'].value_counts()

https://www.doordash.com/store/7552/?pickup=false        3
https://www.doordash.com/store/8902/?pickup=false        3
https://www.doordash.com/store/619123/?pickup=false      3
https://www.doordash.com/store/6843/?pickup=false        3
https://www.doordash.com/store/179523/?pickup=false      3
                                                        ..
https://www.doordash.com/store/22926012/?pickup=false    1
https://www.doordash.com/store/1516242/?pickup=false     1
https://www.doordash.com/store/1709512/?pickup=false     1
https://www.doordash.com/store/2813281/?pickup=false     1
https://www.doordash.com/store/2299240/?pickup=false     1
Name: url, Length: 4078, dtype: int64

In [70]:
states = ['AL',
'AK',
'AZ',
'AR',
'CO',
'CT',
'DE',
'FL',
'GA',
'HI',
'ID',
'IN',
'IA',
'KS',
'KY',
'LA',
'ME',
'MD',
'MI',
'MN',
'MS',
'MO',
'MT',
'NE',
'NV',
'NH',
'NM',
'NC',
'ND',
'OH',
'OK',
'OR',
'PA',
'RI',
'SC',
'SD',
'TN',
'TX',
'UT',
'VT',
'VA',
'WA',
'WV',
'WI',
'WY']

temp_pd = cleaner_dd.copy()

temp_pd['state split'] = temp_pd['address'].str.split(' ')

all_addresses = temp_pd['state split'].explode()
all_addresses.value_counts().head(25)


all_addresses = all_addresses.reset_index()
all_addresses = all_addresses.drop(columns='index')
all_addresses

# # all_cuisine_types.values
# dd_states = {'cuisine': all_cuisine_types['cuisines split'].value_counts().index, 'count': all_cuisine_types['cuisines split'].value_counts().values}
# dd_states = pd.DataFrame(dd_states)
all_addresses[all_addresses['state split'].isin(states)].value_counts()

state split
FL             6
LA             2
GA             1
ID             1
NE             1
TX             1
WA             1
dtype: int64